In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from utils import *
from xgboost import XGBClassifier

from imblearn.pipeline import Pipeline as ImbPipeline

In [2]:
X = pd.read_csv("TrainningInput_Dataset_myseg_area.csv",index_col = 0)
print(X.shape)

y = pd.read_csv("TrainningOutput_Dataset_myseg_area.csv",index_col=0)
print(y.shape)

X_test = pd.read_csv("TestingInput_Dataset_myseg_area.csv",index_col=0)

# Shuffle the data
X = X.sample(n=X.shape[0])
y = y.loc[X.index]

# Here create other feature from the original ones. Maybe compute body surface, first i did imc but what I really wanted was like the average organ size of a patient so I took instead a measure of body area.. Then remove the feature height and weight.
add_body_surface_area_feature(X)
X.drop(columns=["Height", "Weight"],axis=1,inplace=True)
add_ratio_features(X)

add_body_surface_area_feature(X_test)
X_test.drop(columns=["Height","Weight"],axis =1,inplace = True)
add_ratio_features(X_test)


(100, 14)
(100, 1)
body surface are feature added modified


/Users/rplanchon/Documents/telecom/IMA/S2/IMA205/Challenge/CardiacPathoPrediction/utils.py:75: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_col_name] = df[col1] / df[col2].replace(0, float('nan'))
/Users/rplanchon/Documents/telecom/IMA/S2/IMA205/Challenge/CardiacPathoPrediction/utils.py:75: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_col_name] = df[col1] / df[col2].replace(0, float('nan'))
/Users/rplanchon/Documents/telecom/IMA/S2/IMA205/Challenge/CardiacPathoPrediction/utils.py:75: PerformanceWarning: DataFrame

body surface are feature added modified


/Users/rplanchon/Documents/telecom/IMA/S2/IMA205/Challenge/CardiacPathoPrediction/utils.py:75: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_col_name] = df[col1] / df[col2].replace(0, float('nan'))
/Users/rplanchon/Documents/telecom/IMA/S2/IMA205/Challenge/CardiacPathoPrediction/utils.py:75: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[new_col_name] = df[col1] / df[col2].replace(0, float('nan'))
/Users/rplanchon/Documents/telecom/IMA/S2/IMA205/Challenge/CardiacPathoPrediction/utils.py:75: PerformanceWarning: DataFrame

In [3]:
pipeline = ImbPipeline([
    ("normaliser" , MinMaxScaler()),
    ("dataAugment",GaussianNoiseInjector()),
    ("classifier", RandomForestClassifier()),
])
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'transform_input', 'verbose', 'normaliser', 'dataAugment', 'classifier', 'normaliser__clip', 'normaliser__copy', 'normaliser__feature_range', 'dataAugment__noise_factor', 'dataAugment__random_state', 'classifier__bootstrap', 'classifier__ccp_alpha', 'classifier__class_weight', 'classifier__criterion', 'classifier__max_depth', 'classifier__max_features', 'classifier__max_leaf_nodes', 'classifier__max_samples', 'classifier__min_impurity_decrease', 'classifier__min_samples_leaf', 'classifier__min_samples_split', 'classifier__min_weight_fraction_leaf', 'classifier__monotonic_cst', 'classifier__n_estimators', 'classifier__n_jobs', 'classifier__oob_score', 'classifier__random_state', 'classifier__verbose', 'classifier__warm_start'])

In [4]:
param_grid = {
    'dataAugment__noise_factor' : [0.05,0.0],
    'classifier__n_estimators': [100,300,500,1000],
    'classifier__max_features': ['sqrt'],
    'classifier__max_depth': [5,15],
    'classifier__min_samples_split': [2,5],
    'classifier__min_samples_leaf': [2,5],
}

In [5]:
grid_search = GridSearchCV(pipeline,param_grid=param_grid,cv=5,verbose=3,return_train_score=True)
grid_search.fit(X,y["Category"])

Fitting 5 folds for each of 64 candidates, totalling 320 fits
[CV 1/5] END classifier__max_depth=5, classifier__max_features=sqrt, classifier__min_samples_leaf=2, classifier__min_samples_split=2, classifier__n_estimators=100, dataAugment__noise_factor=0.05;, score=(train=1.000, test=0.950) total time=   0.9s
[CV 2/5] END classifier__max_depth=5, classifier__max_features=sqrt, classifier__min_samples_leaf=2, classifier__min_samples_split=2, classifier__n_estimators=100, dataAugment__noise_factor=0.05;, score=(train=0.988, test=0.950) total time=   0.3s
[CV 3/5] END classifier__max_depth=5, classifier__max_features=sqrt, classifier__min_samples_leaf=2, classifier__min_samples_split=2, classifier__n_estimators=100, dataAugment__noise_factor=0.05;, score=(train=0.988, test=1.000) total time=   0.6s
[CV 4/5] END classifier__max_depth=5, classifier__max_features=sqrt, classifier__min_samples_leaf=2, classifier__min_samples_split=2, classifier__n_estimators=100, dataAugment__noise_factor=0.05

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('normaliser', MinMaxScaler()),
                                       ('dataAugment', GaussianNoiseInjector()),
                                       ('classifier',
                                        RandomForestClassifier())]),
             param_grid={'classifier__max_depth': [5, 15],
                         'classifier__max_features': ['sqrt'],
                         'classifier__min_samples_leaf': [2, 5],
                         'classifier__min_samples_split': [2, 5],
                         'classifier__n_estimators': [100, 300, 500, 1000],
                         'dataAugment__noise_factor': [0.05, 0.0]},
             return_train_score=True, verbose=3)

The model is fitted and now we want to properly evaluate the results. 
We select only the result from the best_params founded by the search. (Reminder that the best params are the one that provided the best mean validation score)

In [6]:
best_params = grid_search.best_params_
print(best_params)
results = grid_search.cv_results_

# Only keep the cv result for the best paramaters.
best_idx = results['params'].index(best_params)

mean_train_score = results['mean_train_score'][best_idx]
mean_valid_score = results['mean_test_score'][best_idx]
std_train_score = results['std_train_score'][best_idx]
std_valid_score = results['std_test_score'][best_idx]
print(f"score validation set : {mean_valid_score,std_valid_score}")
print(f"score trainning set : {mean_train_score,std_train_score}")

{'classifier__max_depth': 5, 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf': 2, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 500, 'dataAugment__noise_factor': 0.05}
score validation set : (np.float64(0.93), np.float64(0.0748331477354788))
score trainning set : (np.float64(0.99), np.float64(0.004999999999999982))


In [7]:
# CONSTRUCTION DE LA PIPELINE D"INFERENCE QUI NE FAIT PLUS DE DATA AUG

best_pipeline = grid_search.best_estimator_
inference_steps = [
    (name, step)
    for name, step in best_pipeline.steps
    if name != "dataAugment"
]
inf_pipeline = Pipeline(inference_steps)
print(inf_pipeline.get_params())

{'memory': None, 'steps': [('normaliser', MinMaxScaler()), ('classifier', RandomForestClassifier(max_depth=5, min_samples_leaf=2, n_estimators=500))], 'transform_input': None, 'verbose': False, 'normaliser': MinMaxScaler(), 'classifier': RandomForestClassifier(max_depth=5, min_samples_leaf=2, n_estimators=500), 'normaliser__clip': False, 'normaliser__copy': True, 'normaliser__feature_range': (0, 1), 'classifier__bootstrap': True, 'classifier__ccp_alpha': 0.0, 'classifier__class_weight': None, 'classifier__criterion': 'gini', 'classifier__max_depth': 5, 'classifier__max_features': 'sqrt', 'classifier__max_leaf_nodes': None, 'classifier__max_samples': None, 'classifier__min_impurity_decrease': 0.0, 'classifier__min_samples_leaf': 2, 'classifier__min_samples_split': 2, 'classifier__min_weight_fraction_leaf': 0.0, 'classifier__monotonic_cst': None, 'classifier__n_estimators': 500, 'classifier__n_jobs': None, 'classifier__oob_score': False, 'classifier__random_state': None, 'classifier__ver

In [8]:
from sklearn.metrics import confusion_matrix,accuracy_score

y_pred = inf_pipeline.predict(X)
cm = confusion_matrix(y,y_pred)
acc = accuracy_score(y,y_pred)
print(acc)
print(cm)

0.99
[[20  0  0  0  0]
 [ 0 19  1  0  0]
 [ 0  0 20  0  0]
 [ 0  0  0 20  0]
 [ 0  0  0  0 20]]


In [9]:
proba = inf_pipeline.predict_proba(X)
L = [0,0,0,0,0]
for x in proba : 
    a= 0
    index = []
    for ind,i in enumerate(x) :
        if i > 0.38 : 
            a+=1
            index.append(ind)
    if a >=2 : 
        for j in index : 
            L[j] +=1  
print(proba)  
print(L)

[[9.28571429e-03 3.00000000e-03 6.66666667e-04 9.87047619e-01
  0.00000000e+00]
 [8.54329004e-03 8.51892419e-01 7.19035600e-02 5.36514816e-02
  1.40092492e-02]
 [1.66666667e-03 1.39463577e-01 8.43903206e-01 1.57777778e-03
  1.33887730e-02]
 [9.53820202e-01 9.09259259e-03 6.66666667e-04 2.79575758e-02
  8.46296296e-03]
 [1.62797980e-01 5.14814815e-03 3.60000000e-03 7.42424242e-03
  8.21029630e-01]
 [7.15222222e-02 1.00481481e-02 5.00000000e-03 4.76666667e-02
  8.65762963e-01]
 [8.12265512e-03 8.48995721e-01 1.38405020e-01 1.85783085e-03
  2.61877303e-03]
 [9.04235888e-01 2.39657952e-02 2.66666667e-03 4.12020202e-02
  2.79296296e-02]
 [2.72337662e-02 6.66921287e-01 2.89256751e-01 1.33306600e-02
  3.25753623e-03]
 [9.78757576e-02 1.43333333e-02 1.00000000e-03 8.54557576e-01
  3.22333333e-02]
 [8.04559885e-02 8.34663405e-01 4.91706698e-02 2.74578309e-02
  8.25210637e-03]
 [7.81666667e-02 3.28000000e-02 4.33333333e-02 4.46666667e-03
  8.41233333e-01]
 [7.00000000e-03 4.88888889e-03 6.666666

In [10]:
submission_name = "submission_9.csv"
submission_dataframe = pd.DataFrame(columns=["Id","Category"])
submission_dataframe["Id"] = X_test.index + 101
y_test_pred = inf_pipeline.predict(X_test)
submission_dataframe["Category"] = y_test_pred
submission_dataframe.to_csv(os.path.join(os.getcwd(),submission_name),index=False)
print("file saved")

file saved


In [20]:
print(results)

{'mean_fit_time': array([0.45309234, 0.3221806 , 0.2876852 , 0.26943736, 0.34965186,
       0.30838156, 0.3356122 , 0.47582202, 0.78869152, 0.93373723,
       0.86713185, 1.18293099, 1.01818452, 0.78337879, 0.78320885,
       1.08181181, 1.2430222 , 1.23056765, 1.49540677, 1.45445552,
       1.49117894, 1.2536696 , 1.23360205, 1.57862501]), 'std_fit_time': array([0.28459516, 0.0151199 , 0.02373994, 0.00373843, 0.09258465,
       0.05018674, 0.01295467, 0.16374434, 0.12495868, 0.1145843 ,
       0.08713594, 0.34805971, 0.36612416, 0.03662771, 0.03888772,
       0.39483468, 0.01911271, 0.01015333, 0.31840429, 0.24925409,
       0.23196023, 0.01827378, 0.01057265, 0.46422141]), 'mean_score_time': array([0.02025881, 0.01911902, 0.01887088, 0.0182241 , 0.05013237,
       0.021106  , 0.02037144, 0.04225116, 0.04688945, 0.04542885,
       0.05473356, 0.04943643, 0.04500446, 0.0502439 , 0.04242992,
       0.05279126, 0.07149911, 0.0725842 , 0.0720274 , 0.12420115,
       0.12557187, 0.07177119

In [11]:
f_importance = grid_search.best_estimator_.named_steps["classifier"].feature_importances_
f_name = grid_search.best_estimator_.named_steps["normaliser"].get_feature_names_out()
feature_importance = pd.DataFrame({
    "feature": f_name,
    "importance": f_importance
})
feature_importance.sort_values("importance", ascending=False, inplace=True)
print(feature_importance)

                           feature  importance
20         ED_RV_vol_div_ES_LV_vol    0.028274
94         ES_RV_vol_div_ES_MY_vol    0.028065
109        ES_LV_vol_div_ED_RV_vol    0.027842
44         ED_LV_vol_div_ES_LV_vol    0.027359
118        ES_LV_vol_div_ES_MY_vol    0.027125
..                             ...         ...
27   ED_RV_border_div_ED_LV_border    0.000270
97      ES_RV_border_div_ED_RV_vol    0.000202
36   ED_RV_border_div_body_surface    0.000165
77      ED_MY_border_div_ED_MY_vol    0.000130
51      ED_LV_border_div_ED_LV_vol    0.000119

[169 rows x 2 columns]


Below two cells to save the results : the model and the description of the method used.

In [16]:
description = "Data augmentation cleaned + MinmaxScaler + Randomforest with Gridsearch." 
other_params = "The features are just the volume of each segmentation + body surface + all the possible ratios."
name_folder = "RF_data_aug_noise_pipeline"
feature_used = f_name
informationDict = {
    "description": description,
    "model parameters" : best_params,
    "features used" : feature_used,
    "mean test accuracy with best params" : mean_valid_score ,
    "std  test with best params" : std_valid_score,
    "mean train accuracy with best params" : mean_train_score,
    "std train best params" : std_train_score,
    "other parms" : other_params,
    
}

In [17]:
import joblib
from datetime import datetime
import os

# Create a timestamp
currentDateTime = datetime.now()

# Get the base directory (current directory)
BASE_DIR = os.getcwd()
print(f"BASE_DIR: {BASE_DIR}")
RESULT_DIR = os.path.join(BASE_DIR,"output")
if not os.path.exists(RESULT_DIR):
    os.mkdir(RESULT_DIR)
    print(f"Directory created: {RESULT_DIR}")

# Create a folder named 'pipeline_<timestamp>' in the current directory
dir_name = name_folder
dir_path = os.path.join(RESULT_DIR, dir_name)

# If the directory doesn't exist, create it
if not os.path.exists(dir_path):
    os.mkdir(dir_path)
    print(f"Directory created: {dir_path}")

# Save the model inside this new folder
model_filename = 'pipeline_' + currentDateTime.strftime("%H-%M-%S") + '.pkl'
model_path = os.path.join(dir_path, model_filename)

# This is where you'd have your model defined
joblib.dump(grid_search, model_path)
print(f"Model saved to: {model_path}")


# Saving feature importance : 
feature_importance_filename ='pipeline_' + currentDateTime.strftime("%H-%M-%S") + '.csv' 
feature_importance_dir = os.path.join(dir_path,feature_importance_filename)
feature_importance.to_csv(feature_importance_dir)


# SAVING Description 
dict_filename = 'params.txt'
dict_path = os.path.join(dir_path, dict_filename)

with open(dict_path, 'w') as f:
    for key, val in informationDict.items():
        f.write(f"{key} : {val}\n")
print(f"Information about the model saved to: {dict_path}")


BASE_DIR: /Users/rplanchon/Documents/telecom/IMA/S2/IMA205/Challenge/CardiacPathoPrediction


Model saved to: /Users/rplanchon/Documents/telecom/IMA/S2/IMA205/Challenge/CardiacPathoPrediction/output/RF_data_aug_noise_pipeline/pipeline_01-08-15.pkl
Information about the model saved to: /Users/rplanchon/Documents/telecom/IMA/S2/IMA205/Challenge/CardiacPathoPrediction/output/RF_data_aug_noise_pipeline/params.txt
